**Task 1: Pre-process the dataset**

Read in the csv data.

In [1]:
import csv
import numpy as numpy
import pandas as pd

In [4]:
with open('dataset_mood_smartphone.csv', 'r') as csvFile:
    #reader = csv.reader(csvFile)
    reader = csv.DictReader(csvFile)
    #next(reader)
    #for row in reader:
        
        #print(row)

   # csvFile.close()

TypeError: 'DictReader' object does not support indexing

In [6]:
df = pd.read_csv("dataset_mood_smartphone.csv")
counted = df.groupby(["id"]).size()
print (len(counted))
print (counted)

27
id
AS14.01    21999
AS14.02    14581
AS14.03    14425
AS14.05    15745
AS14.06    18092
AS14.07    16045
AS14.08     7902
AS14.09    10886
AS14.12    17311
AS14.13    19592
AS14.14     9286
AS14.15     2848
AS14.16     3982
AS14.17    15826
AS14.19    11397
AS14.20     3620
AS14.23    21852
AS14.24    14430
AS14.25    12589
AS14.26    16403
AS14.27    14575
AS14.28    19276
AS14.29    17499
AS14.30    17279
AS14.31    11889
AS14.32    11193
AS14.33    16390
dtype: int64


Making individual files for all patients

In [7]:
RawFile = "dataset_mood_smartphone.csv"

with open(RawFile) as infile:
    reader = csv.reader(infile)
    input_header = reader.next()
    for row in reader:
        patient = row[1]
        #print(mmsi)
        CleanedFile= patient +".csv"
        with open(CleanedFile, 'a') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(row)